In [7]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

# Your CryptoPanic API key
api_key = "8e28de32e82845fc3c95c005932eb8ed2fc67f87"
base_url = f"https://cryptopanic.com/api/v1/posts/?auth_token={api_key}&kind=news"

# Set up Selenium WebDriver options
options = Options()
options.add_argument("--headless")  # Run in headless mode (no UI)
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
)

# Initialize the WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Fetch posts from the first 5 pages
def fetch_posts_from_multiple_pages(pages=5):
    all_posts = []
    
    for page in range(1, pages + 1):
        url = f"{base_url}&page={page}"
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            posts = data.get("results", [])

            if not posts:
                print(f"No posts found on page {page}.")
                break

            all_posts.extend(posts)
        else:
            print(f"Error: Unable to fetch posts for page {page} (status code: {response.status_code})")
            break

    return all_posts

posts = fetch_posts_from_multiple_pages(pages=1)

# Loop through the posts and scrape the article content
for i, post in enumerate(posts): 
    title = post["title"]
    link = post["url"]
    publish_date = post["published_at"]
    currency = post.get("currencies", [])  # Fix: Avoid KeyError
    currency_codes = [c["code"] for c in currency]  # Extract codes safely
    
    print(f"{i+1}. Title: {title}")

    # Open the article URL in Selenium
    driver.get(link)
    time.sleep(0.5)  # Wait for the page to load
    
    # Try to find the description-body content
    try:
        content_element = driver.find_element(By.CLASS_NAME, "description-body")
        content = content_element.text
        with open("cryptopanic.txt", "a", encoding="utf-8") as file:
            file.write(f"title: {title}"+"\n")
            file.write(f"Content: {content}"+"\n")
            file.write(f"Publish Date: {publish_date}"+"\n")
            file.write(f"Currencies: {', '.join(currency_codes) if currency_codes else 'None'}"+"\n")
            file.write("=" * 50 + "\n")
    except Exception as e:
        print(f"   Error scraping content: {e}")
    
    print("\n")

# Close the WebDriver
driver.quit()


1. Title: Solana drops 40% – Can it recover before March’s token unlock?


2. Title: Tea-Fi integrates Orbs’ dLIMIT and dTWAP protocols


3. Title: Reserve Bank of Australia (RBA) cuts interest rates for the first time since 2020


4. Title: Dubai Regulator Mandates Memecoins Follow Regulations


5. Title: Hyperliquid rolls out HyperEVM, unveils bug bounty rewards


6. Title: Bitcoin Bull Run Just Starting as Firms Buy, Bernstein Says


7. Title: Ethereum (ETH) Price May Record Big Rebound Soon, Here's Why


8. Title: Pi Network: Binance Launches Community Vote For Listing


9. Title: Tether signs blockchain adoption agreement with Guinea to drive tech innovation


10. Title: Hedera (HBAR) Faces Market Turbulence as Traders Waver Between Gains and Losses


11. Title: SEC’s Crypto Task Force to Build National Bitcoin Reserve: Bullish for $BTCBULL?


12. Title: LIBRA, Solana, and Melania discussions dominate social media buzz


13. Title: Argentina president Javier Milei claims LIBRA mem